# Pauta Tarea 2

## Librerías

In [1]:
import math
import numpy as np
import pandas as pd
from scipy.stats import norm
from dataclasses import dataclass
from typing import Iterable, Tuple
from scipy.optimize import minimize

import modules.hull_white as hw

## Importa Data

In [2]:
data = pd.read_excel(
    'data_enunciado_tarea_2.xlsx',
    sheet_name='Sheet1',
    usecols='A:D',
)
data.columns = ['maturity', 'tasa_swap', 'df', 'cap_vol']

Formato para el `DataFrame`.

In [3]:
frmt = {
    'maturity': '{:.2f}',
    'tasa_swap': '{:.4%}',
    'df': '{:.8%}',
    'cap_vol': '{:.5%}'
}

Se visualiza.

In [4]:
data.style.format(frmt)

,maturity,tasa_swap,df,cap_vol
0,0.25,2.1800%,99.45795415%,nan%
1,0.50,2.3177%,98.85094864%,0.04560%
2,0.75,2.4420%,98.18987496%,0.10590%
3,1.00,2.5550%,97.48343859%,0.18590%
4,1.25,2.6586%,96.73842526%,0.28870%
5,1.50,2.7546%,95.95982497%,0.41570%
6,1.75,2.8451%,95.15030816%,0.56620%
7,2.00,2.9320%,94.31088166%,0.73640%
8,2.25,3.0167%,93.44181883%,0.92010%
9,2.50,3.0991%,92.54566396%,1.11290%


## Modelo para un Cap y sus Caplets

### Clase `Caplet`

In [5]:
@dataclass
class Caplet:
    notional: float
    start_time: float
    end_time: float
    strike: float
        
    def get_yf(self):
        return self.end_time - self.start_time

### Alias `Cap`

In [6]:
Cap = Iterable[Caplet]

### Factory Function

In [7]:
def make_cap(
    strike: float,
    num_caplets: int,
    periodicity: float,
    notional: float
) -> Cap:
    """
    Construye un Cap.
    
    - strike: strike del Cap.
    - num_caplets: número de caplets. Debe incluir el caplet que comienza en `t = 0`.
    - periodicity: periodicidad de los caplets (equivalente al tenor del índice). Se expresa en fracción
    de año canónica, por ejemplo 3M -> 0.25, 6M -> .50.
    - notional: nocional del CAP.
    """
    result = []
    for i in range(num_caplets):
        start_time = i * periodicity
        end_time = (i + 1) * periodicity
        result.append(
            Caplet(
                notional=notional,
                start_time=start_time,
                end_time=end_time,
                strike=strike
            )
        )
    return result

In [8]:
test_cap = make_cap(
    strike=.026,
    num_caplets=4,
    periodicity=.25,
    notional=10000000
)

In [9]:
test_cap

[Caplet(notional=10000000, start_time=0.0, end_time=0.25, strike=0.026),
 Caplet(notional=10000000, start_time=0.25, end_time=0.5, strike=0.026),
 Caplet(notional=10000000, start_time=0.5, end_time=0.75, strike=0.026),
 Caplet(notional=10000000, start_time=0.75, end_time=1.0, strike=0.026)]

## Modelo para la Curva Cero

Se define una clase que contenga los factores de descuento de la curva cero cupón. La clase define un método de interpolación log-lineal en dichos factores de descuento.

In [10]:
@dataclass
class DiscountFactorCurve:
    tenors_dfs: Iterable[Tuple[float, float]]

    def __post_init__(self):
        self.tenors_dfs.sort(key=lambda x: x[0])

    def get_df_at(self, t: float) -> float:
        return math.exp(
            np.interp(
                t,
                xp=[x[0] for x in self.tenors_dfs],
                fp=[math.log(x[1]) for x in self.tenors_dfs],
            ))

    def get_linear_rate_at(self, t: float) -> float:
        df = self.get_df_at(t)
        return (1 / df - 1) / t
    
    def get_continous_rate_at(self, t: float) -> float:
        df = self.get_df_at(t)
        return -math.log(df) / t
        
    def get_df_fwd_between(self, t0: float, t1: float) -> float:
        return self.get_df_at(t1) / self.get_df_at(t0)

    def get_linear_fwd_rate_between(self, t0: float, t1: float) -> float:
        df_fwd = self.get_df_fwd_between(t0, t1)
        return (1 / df_fwd - 1) / (t1 - t0)

Se obtiene la data y se da de alta el objeto.

In [11]:
tenors_dfs = [(x[0], x[2]) for x in data.values]

In [12]:
tenors_dfs

[(0.25, 0.9945795414988314),
 (0.5, 0.9885094864000967),
 (0.75, 0.9818987496182575),
 (1.0, 0.9748343859309748),
 (1.25, 0.9673842525560281),
 (1.5, 0.9595982496698358),
 (1.75, 0.9515030815707846),
 (2.0, 0.943108816587112),
 (2.25, 0.9344181882584228),
 (2.5, 0.925456639619877),
 (2.75, 0.9162685487133939),
 (3.0, 0.9068993238848506),
 (3.25, 0.8973959863734557),
 (3.5, 0.8877796221908567),
 (3.75, 0.8780496480797562),
 (4.0, 0.8682121096506916),
 (4.25, 0.858264465488608),
 (4.5, 0.8482195304644343),
 (4.75, 0.8381019804697758),
 (5.0, 0.8279376886770442)]

### Se Prueban los Métodos

No se trata de test unitarios tradicionales ya que los resultados no se comparan con resultados esperados.

In [13]:
df_curve = DiscountFactorCurve(tenors_dfs)
print(df_curve.get_df_at(0.0))
print(df_curve.get_linear_rate_at(.25))
print(df_curve.get_df_fwd_between(0.0, .25))
print(df_curve.get_linear_fwd_rate_between(1.0, 2.0))

0.9945795414988314
0.02179999999999982
1.0
0.03363935188165268


## Funciones de Valorización

Las siguientes funciones permiten valorizar un caplet, y en consecuencia un cap, usando la volatilidad de mercado y la fórmula de Black-Scholes-Merton.

In [14]:
def value_caplet(
    caplet: Caplet,
    dfs: DiscountFactorCurve,
    vol: float,
) -> float:
    if caplet.start_time < 0.0:
        return 0.0
    elif caplet.start_time == 0.0:
        yf = caplet.get_yf()
        fwd_rate = dfs.get_linear_fwd_rate_between(
            caplet.start_time,
            caplet.end_time
        )
        return caplet.notional * yf * max(fwd_rate - caplet.strike, 0.0)
    else:
        yf = caplet.get_yf()
        fwd_rate = dfs.get_linear_fwd_rate_between(
            caplet.start_time,
            caplet.end_time
        )
        df = dfs.get_df_at(caplet.end_time)
        sigma_sqrT = vol * math.sqrt(caplet.end_time)
        d1 = (math.log(
                fwd_rate / caplet.strike) + .5 * sigma_sqrT**2
        ) / sigma_sqrT
        d2 = d1 - sigma_sqrT
        black = fwd_rate * norm.cdf(d1) - caplet.strike * norm.cdf(d2)
        return caplet.notional * yf * df * black

In [15]:
value_caplet(test_cap[1], df_curve, .1)

526.5648800994534

In [16]:
def value_cap(
    cap: Cap,
    dfs: DiscountFactorCurve,
    vol: float
) -> float:
    return sum([value_caplet(caplet, dfs, vol) for caplet in cap])

Se construyen todos los caps.

In [17]:
all_caps = []
for i in range(1, len(data)):
    all_caps.append(
        make_cap(
            strike=data.iloc[i].tasa_swap,
            num_caplets=i + 1,
            periodicity=.25,
            notional=10000000
        )
    )

Se visualiza un Cap en particular.

In [18]:
which_cap = 2
all_caps[which_cap]

[Caplet(notional=10000000, start_time=0.0, end_time=0.25, strike=0.02555),
 Caplet(notional=10000000, start_time=0.25, end_time=0.5, strike=0.02555),
 Caplet(notional=10000000, start_time=0.5, end_time=0.75, strike=0.02555),
 Caplet(notional=10000000, start_time=0.75, end_time=1.0, strike=0.02555)]

Se valoriza cada Cap y se muestra el resultado. La valorización utiliza la volatilidad entregada en la data.

In [19]:
for which_cap, cap in enumerate(all_caps):
    print(f'Valor de mercado del cap {which_cap + 1}-esimo: \
    {value_cap(cap, df_curve, data.iloc[which_cap + 1].cap_vol):,.0f}')

Valor de mercado del cap 1-esimo:     3,424
Valor de mercado del cap 2-esimo:     6,514
Valor de mercado del cap 3-esimo:     11,765
Valor de mercado del cap 4-esimo:     16,901
Valor de mercado del cap 5-esimo:     23,171
Valor de mercado del cap 6-esimo:     29,880
Valor de mercado del cap 7-esimo:     37,146
Valor de mercado del cap 8-esimo:     45,481
Valor de mercado del cap 9-esimo:     54,230
Valor de mercado del cap 10-esimo:     63,832
Valor de mercado del cap 11-esimo:     73,737
Valor de mercado del cap 12-esimo:     83,977
Valor de mercado del cap 13-esimo:     94,418
Valor de mercado del cap 14-esimo:     105,113
Valor de mercado del cap 15-esimo:     116,069
Valor de mercado del cap 16-esimo:     127,336
Valor de mercado del cap 17-esimo:     138,898
Valor de mercado del cap 18-esimo:     150,699
Valor de mercado del cap 19-esimo:     162,684


## Funciones de Valorización HW

In [20]:
def value_caplet_hw(
    caplet: Caplet,
    dfs: DiscountFactorCurve,
    gamma: float,
    sigma: float,
) -> float:
    if caplet.start_time < 0.0:
        return 0.0
    elif caplet.start_time == 0.0:
        yf = caplet.get_yf()
        fwd_rate = dfs.get_linear_fwd_rate_between(
            caplet.start_time,
            caplet.end_time
        )
        return caplet.notional * yf * max(fwd_rate - caplet.strike, 0.0)
    else:
        yf = caplet.get_yf()
        new_strike = 1 / (1 + caplet.strike * yf)
        fwd_rate = dfs.get_linear_fwd_rate_between(
            caplet.start_time,
            caplet.end_time
        )
        new_notional = caplet.notional * (1 + caplet.strike * yf)
        zo = dfs.get_df_at(caplet.start_time)
        zb = dfs.get_df_at(caplet.end_time)
        sz = hw.sz(
            caplet.start_time,
            caplet.end_time,
            gamma,
            sigma
        )
        r0 = dfs.get_continous_rate_at(
            dfs.tenors_dfs[0][0]
        )
        # print(new_strike, zb/zo, r0)
        return new_notional * hw.zcb_call_put(
            hw.CallPut.PUT,
            new_strike,
            caplet.start_time,
            caplet.end_time,
            r0,
            zo,
            zb,
            gamma,
            sigma,
        )

In [21]:
def value_cap_hw(
    cap: Cap,
    dfs: DiscountFactorCurve,
    gamma: float,
    sigma: float,
) -> float:
    return sum([value_caplet_hw(caplet, dfs, gamma, sigma) for caplet in cap])

In [22]:
for which_cap, cap in enumerate(all_caps):
    print(f'Valor de modelo del cap {which_cap + 1}-esimo: \
    {value_cap_hw(cap, df_curve, .5, .005):,.0f}')

Valor de modelo del cap 1-esimo:     4,317
Valor de modelo del cap 2-esimo:     9,342
Valor de modelo del cap 3-esimo:     15,169
Valor de modelo del cap 4-esimo:     21,746
Valor de modelo del cap 5-esimo:     28,972
Valor de modelo del cap 6-esimo:     36,803
Valor de modelo del cap 7-esimo:     45,261
Valor de modelo del cap 8-esimo:     54,399
Valor de modelo del cap 9-esimo:     64,165
Valor de modelo del cap 10-esimo:     74,411
Valor de modelo del cap 11-esimo:     84,979
Valor de modelo del cap 12-esimo:     95,700
Valor de modelo del cap 13-esimo:     106,527
Valor de modelo del cap 14-esimo:     117,517
Valor de modelo del cap 15-esimo:     128,688
Valor de modelo del cap 16-esimo:     140,100
Valor de modelo del cap 17-esimo:     151,732
Valor de modelo del cap 18-esimo:     163,508
Valor de modelo del cap 19-esimo:     175,344


In [23]:
def error(gamma_sigma: Iterable[float], *args) -> float:
    caps = args[0]
    dfs = args[1]
    data = args[2]
    result = 0.0
    for i, cap in enumerate(caps):
        result += (value_cap(cap, dfs, data.iloc[i + 1].cap_vol) - 
                   value_cap_hw(cap, dfs, gamma_sigma[0], gamma_sigma[1])) ** 2
    return result

In [24]:
gs0 = [.5, .005]
error(gs0, all_caps, df_curve, data)

1856995607.8310335

## Encuentra `gamma`y `sigma`

In [25]:
res = minimize(
    error,
    gs0,
    args=(all_caps, df_curve, data),
    # method='Powell',
    bounds = [(0.0001, None), (0.0001, None)],
    tol=1e-8
)

In [26]:
res

      fun: 6256168.711228065
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([7325758.22621963,   -9341.91048151])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 63
      nit: 18
     njev: 21
   status: 0
  success: True
        x: array([0.0001    , 0.00097907])